# Exemplo Árvore de decisão (regressão) com Cluster

Neste exemplo será utilizado os dados da pesquisa sobre *Nerdy Personality Attributes Scale*

## Carregar pacotes

In [ ]:
library(tidyverse)
library(magrittr)
library(rpart)
library(clue)
library(plotly)

## Funções gerais

In [ ]:
rmse <- function( real, prediction ){ sqrt( mean( (real - prediction)^2 ) ) }

In [ ]:
padronizar <- function(x){ (x - min(x)) / (max(x) - min(x)) }

## Carregar dados

In [ ]:
load( '/home/vm-data-science/dados/dados_nerd_nomissings.RData' )

In [ ]:
dados_nerd_train %>% 
    head()

In [ ]:
dados_nerd_test %>% 
    head()

## Análise exploratória

In [ ]:
# count em todas as colunas
dados_nerd_train %>% 
  map( ~count(data.frame(x=.x), x) )

In [ ]:
# Histograma
dados_nerd_train %>% 
  ggplot( aes( x = nerdy ) ) +
  geom_histogram()

In [ ]:
# outliers
dados_nerd_train %>% 
  dplyr::select( contains('Q') ) %>% 
  gather( key = variaveis, value = notas ) %>% 
  plot_ly( x = ~variaveis,
           y = ~notas,
           type = 'box' )

## Combinação de técnicas supervisionadas com não supervisionadas

1 - Aplicamos o algoritmo não supervisionado

2 - Geramos os novos atributos no banco de dados

3 - Aplicamos o algoritmo supervisionado com os atributos obtidos pelo não supervisionado

### 1 - Aplicamos o algoritmo de descoberta de *clusters*

- padronizamos as variáveis

In [ ]:
dados_nerd_train_padronizados <- dados_nerd_train %>% 
  dplyr::select( contains('Q') ) %>% 
  mutate_if( is.numeric, padronizar )

- aplicamos algoritmo hierárquico para auxiliar na obtenção do número de grupos

In [ ]:
distancia <- dist( x = dados_nerd_train_padronizados, 
                   method =  'euclidian' )

modelo_cluster_hierarquico <- hclust( distancia,
                                      method = 'ward.D' )

- geramos o gráfico dos coeficientes de aglomeração (vamos considerar uma solução de 4 grupos, mas poderiamos também testar com 3 grupos)

In [ ]:
data_frame( grupos = 31:1 + 1,
            aglomeracao = modelo_cluster_hierarquico$height[872:902] ) %>% 
  plot_ly( x = ~grupos,
           y = ~aglomeracao,
           type = 'scatter',
           mode = 'lines+markers',
           marker = list(size = 10, color = 'red') ) %>% 
  layout( xaxis = list( autorange="reversed") )

- Dendograma (como existem muitos casos, o dendograma será apresentado somente por objetivos educacionais)

In [ ]:
plot(modelo_cluster_hierarquico)
rect.hclust( modelo_cluster_hierarquico, 
             k = 4, 
             border = "red" )

- obtenção dos centróides

In [ ]:
centroides <- dados_nerd_train_padronizados %>% 
  mutate( grupos = cutree(modelo_cluster_hierarquico, 
                          k = 4 ) ) %>% 
  group_by( grupos ) %>% 
  summarise_if( is.numeric, mean )

- aplicamos o algoritmo não hierárquico

In [ ]:
set.seed(123)
clusters_k4 <- kmeans(dados_nerd_train_padronizados , 
                      centers = centroides[-1] )

### 2 - Geramos os novos atributos no banco de dados

In [ ]:
dados_nerd_train %<>% 
  mutate( grupos_final = as.factor(clusters_k4$cluster) )

In [ ]:
# exemplo
dados_nerd_train %>% 
  dplyr::select( nerdy, Q1:Q5, grupos_final) %>% 
  head()

### 3 - Aplicamos o modelo de árvore de regressão

- Alguns ajustes importantes no banco de dados no R

In [ ]:
dados_nerd_train %<>% 
  mutate( gender = as.factor(gender),
          education = as.factor(education),
          married = as.factor(married),
          ASD = as.factor(ASD) )

- Dividir a amostra de treinamento em: treino/validação

In [ ]:
# treino
set.seed(543)
dados_nerd_train_modelo_train <- dados_nerd_train %>% 
  sample_frac(., 0.8)

In [ ]:
# validacao
dados_nerd_train_modelo_valid <- setdiff( dados_nerd_train, dados_nerd_train_modelo_train )

- Ajustes de poda para vários modelos de árvore em sequência

Usamos a amostra de treino para ajustar o modelo e a amostra de validação para selecionar os melhores e realizar o teste final na amostra de teste

In [ ]:
iteracoes <- 150 # numero de iteracoes para tunning
s_seeds <- sample(1000000:9999999, iteracoes) # sementes aleatorias
dados_amostra_avaliacao_questoes <- NULL
dados_amostra_avaliacao_clust <- NULL

In [ ]:
for ( iter in 1:iteracoes ){
  
  set.seed( s_seeds[iter] )
  minsplit_ <- sample(10:30, 1)
  cp_ <- runif(1, 0.001, 0.1)
  maxcompete_ <- sample(3:30, 1)
  maxdepth_ <- sample(10:50, 1)
  
  modelo_arvore_questoes <- rpart( formula = nerdy ~ ., 
                          data = dados_nerd_train_modelo_train %>% 
                            dplyr::select( nerdy, contains('Q') ),
                          control = list( minsplit = minsplit_,
                                          cp = cp_,
                                          maxcompete = maxcompete_,
                                          maxdepth = maxdepth_ ) )
  
  modelo_arvore_clust <- rpart( formula = nerdy ~ ., 
                                   data = dados_nerd_train_modelo_train %>% 
                                     dplyr::select( nerdy, contains('Q'), grupos_final ),
                                   control = list( minsplit = minsplit_,
                                                   cp = cp_,
                                                   maxcompete = maxcompete_,
                                                   maxdepth = maxdepth_ ) )
  
  pred_tree_questoes <- predict( modelo_arvore_questoes, dados_nerd_train_modelo_valid )
  pred_tree_clust <- predict( modelo_arvore_clust, dados_nerd_train_modelo_valid )
  
  erro_questoes <- rmse( dados_nerd_train_modelo_valid$nerdy, pred_tree_questoes )
  erro_clust <- rmse( dados_nerd_train_modelo_valid$nerdy, pred_tree_clust )
  
  aval_questoes <- data_frame( seed = s_seeds[iter],
                      minsplit_ = minsplit_,
                      cp_ = cp_,
                      maxcompete_ = maxcompete_,
                      maxdepth_ = maxdepth_,
                      erro = erro_questoes )
  
  aval_clust <- data_frame( seed = s_seeds[iter],
                               minsplit_ = minsplit_,
                               cp_ = cp_,
                               maxcompete_ = maxcompete_,
                               maxdepth_ = maxdepth_,
                               erro = erro_clust )
  
  dados_amostra_avaliacao_questoes <- bind_rows( dados_amostra_avaliacao_questoes, aval_questoes )
  dados_amostra_avaliacao_clust <- bind_rows( dados_amostra_avaliacao_clust, aval_clust )
  
}

In [ ]:
bests_questoes <- dados_amostra_avaliacao_questoes %>% 
  arrange( erro ) %>% 
  head(1)

bests_clust <- dados_amostra_avaliacao_clust %>% 
  arrange( erro ) %>% 
  head(1)

In [ ]:
bests_questoes
bests_clust

- Retreina os melhores modelos e guarda resultados

In [ ]:
modelo_arvore_1 <- rpart( formula = nerdy ~ ., 
                        data = dados_nerd_train_modelo_train %>% 
                          dplyr::select( nerdy, contains('Q') ),
                        control = list( minsplit = bests_questoes$minsplit_[1],
                                        cp = bests_questoes$cp_[1],
                                        maxcompete = bests_questoes$maxcompete_[1],
                                        maxdepth = bests_questoes$maxdepth_[1] ) )

In [ ]:
modelo_arvore_2 <- rpart( formula = nerdy ~ ., 
                          data = dados_nerd_train_modelo_train %>% 
                            dplyr::select( nerdy, contains('Q'), grupos_final ),
                          control = list( minsplit = bests_clust$minsplit_[1],
                                          cp = bests_clust$cp_[1],
                                          maxcompete = bests_clust$maxcompete_[1],
                                          maxdepth = bests_clust$maxdepth_[1] ) )

## Avaliação dos modelos

- padronizamos as variáveis da amostra de teste

In [ ]:
dados_nerd_test_padronizados <- dados_nerd_test %>% 
  dplyr::select( contains('Q') ) %>% 
  mutate_if( is.numeric, padronizar )

- criamos os *clusters* para a amostra de teste

In [ ]:
clusters_teste <- cl_predict( clusters_k4, 
            newdata = dados_nerd_test_padronizados )

- Adicionamos os *clusters* na amostra de teste

In [ ]:
dados_nerd_test %<>% 
  mutate( grupos_final = as.factor(clusters_teste) )

In [ ]:
dados_nerd_test %>% 
    head()

- Geramos as previsoes para avaliação

In [ ]:
amostra_avaliacao <- dados_nerd_test %>% 
  mutate( pred_modelo1 = predict( modelo_arvore_1, . ),
          pred_modelo2 = predict( modelo_arvore_2, . ) ) %>% 
  dplyr::select( nerdy, pred_modelo1, pred_modelo2 )

In [ ]:
amostra_avaliacao %>% 
    head()

- Comparamos os modelos pelo RMSE

In [ ]:
amostra_avaliacao %>% 
  summarise( Modelo_1_questoes = rmse(nerdy, pred_modelo1),
             Modelo_2_clusters = rmse(nerdy, pred_modelo2) ) %>% 
  gather( key = modelos, value = rmse ) %>% 
  arrange( rmse )